In [21]:
import base64
from PIL import Image
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
from vertexai.generative_models._generative_models import HarmCategory, HarmBlockThreshold, ResponseBlockedError
from pathlib import Path

vertexai.init(project="crypto-resolver-346012", location="us-central1")

prompt = '''You are an AI expert in creating humour or satire. User gives you an image, and you have to make a choice "A" or "B".
###Instructions: The image is a 2x2 table with the labels "yes" (top left), "but" (top right), "A" (bottom left), and "B" (bottom right). Either the "yes" cell or the "but" cell will have a question mark in it. Your job is to replace the question mark with either cell "A" or cell "B" so that the resulting [yes,but] pair is funny or satirical. Make a choice  "A" or "B":
###Output format: one character, exactly either "A" or "B"'''
def generate(image):
    model = GenerativeModel("gemini-pro-vision")
    
    safety_settings = {
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

    # Yes,But memes are funny because the Yes picture depicts a normal situation, and the But picture reveals something about the Yes picture that makes humans laugh. What is funny about the given Yes,But meme
    
    responses = model.generate_content(
        [prompt,image],
        generation_config={
            "max_output_tokens": 256,
            "temperature": 1,
            "top_p": 1,
            "top_k": 32,
            "candidate_count": 1
        },
        safety_settings=safety_settings,
        
    )
    #print(type(responses))
    #print(responses)
    #print(responses.text)
    return responses.text


with open("yesbut_image_completion/20240101_181415_QMARK_RIGHT.jpg", "rb") as f:
    data = f.read()
    image1 = Part.from_data(data=data, mime_type="image/jpeg")
    #print(image1)

generate(image1)

' A'

In [ ]:
'''
You are an AI expert in creating humour or satire. User gives you an image, and you have to make a choice "A" or "B".

###Structure of image: The image is a 2x2 table with the labels "yes", "but", "A", and "B". Either the "yes" cell or the "but" cell will have a question mark in it. Your job is to replace the question mark with either cell "A" or cell "B" so that the resulting <yes,but> pair is funny or satirical.

###Output format: Option <answer> is more funny because <reason>" where <answer> must be either "A" or "B"

'''

###Output format: " Thus, option <answer> is more funny because <reason>" where <answer> must be either "A" or "B"

"When the question mark is replaced by image A, the resulting [yes,but] pair is <describe it here>.
When the question mark is replaced by image B, the resulting [yes,but] pair is <describe it here>.

In [2]:
import os
def to_dalle_path(type, og_filename, side):
    return os.path.join(f"outputs/dalle3/{type.lower()}/{side.lower()}",f"{og_filename}_{type}_{side}.jpg")

def get_lr_path_from_filename(filename):
    if not "YES" in filename and not "BUT" in filename:
        yes = os.path.join("images_split",f"{filename[:-4]}_YES.jpg")
        but = os.path.join("images_split",f"{filename[:-4]}_BUT.jpg")
    else:
        parts = filename.split("_")
        #print(parts)
        og_filename = "_".join(parts[:-4])
        yestype, buttype = parts[-4], parts[-2]
        yes = to_dalle_path(yestype, og_filename, "YES") if yestype!="ORIG" else os.path.join("images_split",f"{og_filename}_YES.jpg")
        but = to_dalle_path(buttype, og_filename, "BUT") if buttype!="ORIG" else os.path.join("images_split",f"{og_filename}_BUT.jpg")
    return yes,but

print(get_lr_path_from_filename("20240101_172315_STICK_YES_3D_BUT.jpg"))

('outputs/dalle3/stick/yes/20240101_172315_STICK_YES.jpg', 'outputs/dalle3/3d/but/20240101_172315_3D_BUT.jpg')


In [22]:
import os,json
from tqdm import tqdm
from validate import validate_file

outpath = "outputs/completion/gemini-pro-vision-zero.json"
inpath = "yesbut_image_completion"
try:
    with open(outpath, "r") as f:
        outputs = json.load(f)
except FileNotFoundError:
    print("starting from zero")
    outputs = {}

# files = [x for x in os.listdir(inpath) if "BUT" in x]
#redo_files = ["20240101_174949_BUT.jpg"]
## fullfiles = os.listdir(inpath)
## files = set()
## for filename in fullfiles:
##     _,but = get_lr_path_from_filename(filename)
##     if "outputs/dalle3" in but:
##         files.add(but)
files = os.listdir(inpath)
for filename in tqdm(files):
# for filepath in tqdm(files):
#     filename = filepath.split("/")[-1]
#     inpath = "/".join(filepath.split("/")[:-1])
    if not filename[-3:] == "jpg":
        print(f"[WARN] Skipping Weird file: {filename}")
        continue
    if filename in outputs and outputs[filename]:
        continue
    with open(os.path.join(inpath,filename), "rb") as f:
        data = f.read()
        image = Part.from_data(data=data, mime_type="image/jpeg")
    #display(Image.open(os.path.join("images",filename)).convert('RGB'))
    #print(filename)
    try:
        output = generate(image)
    except Exception as e:
        print("Caught exception: ",str(e))
        print("Could not do file: ",filename)
        output = ""
    
    outputs[filename]= output
    with open(outpath, "w") as f:
        json.dump(outputs, f, indent=4)

with open(outpath, "w") as f:
    json.dump(outputs, f, indent=4)
# if not validate_file(outpath):
#     print("Validation failed!")

starting from zero


 18%|█████████████████████████████▋                                                                                                                                        | 27/151 [02:42<11:10,  5.41s/it]

Caught exception:  list index out of range
Could not do file:  20240101_173813_LEFT_QMARK.jpg


 19%|██████████████████████████████▊                                                                                                                                       | 28/151 [02:46<09:53,  4.82s/it]

Caught exception:  list index out of range
Could not do file:  20240101_173239_LEFT_QMARK.jpg


...

 66%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                       | 100/151 [09:49<04:28,  5.26s/it]

Caught exception:  list index out of range
Could not do file:  Screenshot_2024-01-01-17-38-54-117_com.twitter.android-edit_QMARK_RIGHT.jpg


 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                         | 128/151 [12:39<01:59,  5.20s/it]

Caught exception:  list index out of range
Could not do file:  Screenshot_2024-01-01-17-55-08-129_com.twitter.android~2_QMARK_RIGHT.jpg


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151/151 [14:57<00:00,  5.95s/it]


In [ ]:
"""
Caught exception:  500 Internal error encountered.
Could not do file:  20240101_172932.jpg

Caught exception:  500 Internal error encountered.
Could not do file:  20240101_182249.jpg
"""

In [ ]:
"""
manually redid:
["20240101_173205_BUT.jpg", "20240101_173205_YES.jpg", "20240101_174949_BUT.jpg", "20240101_174949_YES.jpg", "20240101_181310_YES.jpg"]
"""

In [20]:
# separate second and third rounds

for task in os.listdir("outputs/second_round"):
    for file in os.listdir(f"outputs/second_round/{task}"):
        if "ipynb" in file:
            continue
        with open(f"outputs/second_round/{task}/{file}", "r") as f:
            outputs = json.load(f)
        third_outs = {}
        og_len = len(outputs)
        for key,val in outputs.items():
            if "3D" in key:
                third_outs[key] = val
        for key in third_outs.keys():
            del outputs[key]
        assert og_len == len(outputs)+len(third_outs)
        with open(f"outputs/second_round/{task}/{file}", "w") as f:
            json.dump(outputs, f, indent=4)
        with open(f"outputs/third_round/{task}/{file}", "w") as f:
            json.dump(third_outs, f, indent=4)